#10

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import keras
import tensorflow as tf
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pandas as pd
import math
import numpy as np
from sklearn.model_selection import train_test_split

# from google.colab import files
# uploaded =  pd.read_excel('/content/drive/MyDrive/ML/data5.xlsx', header = None)

In [9]:
def norm(x):
    return (x - x.mean(axis=0))/x.std(axis=0) 
    # normalization
    
def set(y):
    for i in range(len(y)):
        if(0.0<y[i]<=1.5):
            y[i] = 1.0
        if(1.5<y[i]<=2.5):
            y[i] = 2.0
        if(y[i]>=2.5):
            y[i] = 3.0
    return y

In [13]:
class ANFIS:

    def __init__(self, n_inputs, n_rules, learning_rate=1e-2):
        self.n = n_inputs
        self.m = n_rules
        self.inputs = tf.placeholder(tf.float32, shape=(None, n_inputs))  # Input
        self.targets = tf.placeholder(tf.float32, shape=None)  # Desired output
        mu = tf.get_variable("mu", [n_rules * n_inputs],
                             initializer=tf.random_normal_initializer(0, 1))  # Means of Gaussian MFS
        sigma = tf.get_variable("sigma", [n_rules * n_inputs],
                                initializer=tf.random_normal_initializer(0, 1))  # Standard deviations of Gaussian MFS
        y = tf.get_variable("y", [1, n_rules], initializer=tf.random_normal_initializer(0, 1))  # Sequent centers

        self.params = tf.trainable_variables()

        self.rul = tf.reduce_prod(
            tf.reshape(tf.exp(-0.5 * tf.square(tf.subtract(tf.tile(self.inputs, (1, n_rules)), mu)) / tf.square(sigma)),
                       (-1, n_rules, n_inputs)), axis=2)  # Rule activations
        # Fuzzy base expansion function:
        num = tf.reduce_sum(tf.multiply(self.rul, y), axis=1)
        den = tf.clip_by_value(tf.reduce_sum(self.rul, axis=1), 1e-12, 1e12)
        self.out = tf.divide(num, den)

        self.loss = tf.losses.huber_loss(self.targets, self.out)  # Loss function computation
        self.optimize = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.loss)  # Optimization step
        self.init_variables = tf.global_variables_initializer()  # Variable initializer

    # Function to get predictions from test samples
    def infer(self, sess, x, targets=None):
        if targets is None:
            return sess.run(self.out, feed_dict={self.inputs: x})
        else:
            return sess.run([self.out, self.loss], feed_dict={self.inputs: x, self.targets: targets})

    # Function to initiate and train the graph
    def train(self, sess, x, targets):
        yp, l, _ = sess.run([self.out, self.loss, self.optimize], feed_dict={self.inputs: x, self.targets: targets})
        return l, yp

data = pd.read_excel('/content/drive/MyDrive/ML/data5.xlsx')
data = pd.DataFrame(data)
data = np.asarray(data)
y = data[:,-1]
x = data[:,:-1]
x = norm(x)

x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.3)
m = x_tr.shape[0]
n = x_tr.shape[1]

In [14]:
m = 16  # number of rules
alpha = 0.01  # learning rate
epochs= 2000
fis = ANFIS(n_inputs=7, n_rules=m, learning_rate=alpha)

In [15]:
with tf.Session() as sess:
    # Initialize model parameters
    sess.run(fis.init_variables)
    trn_costs = []
    val_costs = []
    time_start = time.time()
    for epoch in range(epochs):
        # Train the model
        trn_loss, train_pred = fis.train(sess, x_tr, y_tr)
        # Evaluate on test set
        test_pred, val_loss = fis.infer(sess, x_ts, y_ts)
        # Print the training cost
        if epoch % 10 == 0:
            print("Train cost after epoch %i: %f" % (epoch, trn_loss))
        if epoch == epochs - 1:
            time_end = time.time()
            
yp = test_pred # Get the predictions
yp = set(yp)
# Confusion matrix and accuracy
y_actual = pd.Series(y_ts, name='Actual')
y_pred = pd.Series(yp, name='Predicted')
confmat = pd.crosstab(y_actual, y_pred)
print(confmat)

confmat = np.asarray(confmat)
Accuracy = float(confmat[0][0]+confmat[1][1]+confmat[2][2])/float(yp.shape[0])
print('Accuracy :' + ' ' + str(Accuracy))

Train cost after epoch 0: 0.878576
Train cost after epoch 10: 0.643948
Train cost after epoch 20: 0.582066
Train cost after epoch 30: 0.511835
Train cost after epoch 40: 0.441998
Train cost after epoch 50: 0.378444
Train cost after epoch 60: 0.311631
Train cost after epoch 70: 0.250585
Train cost after epoch 80: 0.204925
Train cost after epoch 90: 0.168678
Train cost after epoch 100: 0.139820
Train cost after epoch 110: 0.114149
Train cost after epoch 120: 0.094183
Train cost after epoch 130: 0.078596
Train cost after epoch 140: 0.066125
Train cost after epoch 150: 0.056564
Train cost after epoch 160: 0.049112
Train cost after epoch 170: 0.042473
Train cost after epoch 180: 0.036877
Train cost after epoch 190: 0.032681
Train cost after epoch 200: 0.029682
Train cost after epoch 210: 0.027397
Train cost after epoch 220: 0.025525
Train cost after epoch 230: 0.023813
Train cost after epoch 240: 0.022084
Train cost after epoch 250: 0.020457
Train cost after epoch 260: 0.040480
Train cost a